In [1]:
from ultralytics import YOLO
import os
import random
import cv2
import numpy as np
import shutil
import supervision as sv
import cv2
import matplotlib.pyplot as plt


/home/mbergst/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [ ]:
keypoints = {1: (44, 30), 
             2: (457, 30), 
             3: (44, 563), 
             4: (457, 563), 
             5: (44, 138), 
             6: (174, 138), 
             7: (174, 230), 
             8: (174, 346), 
             9: (174, 453), 
             10: (44, 453), 
             11: (44, 230), 
             12: (86, 230), 
             13: (86, 368), 
             14: (44, 368), 
             15: (130, 295),
             16: (457, 230), 
             17: (457, 295), 
             18: (457, 368), 
             19: (201, 295), 
             20: (387, 295), 
             21: (530, 295), 
             22: (870, 30), 
             23: (870, 563), 
             24: (870, 138), 
             25: (740, 138), 
             26: (740, 230), 
             27: (740, 346), 
             28: (740, 453), 29: (870, 453), 30: (870, 230), 31: (827, 230), 32: (827, 368), 33: (870, 368), 34: (785, 295), 35: (712, 295), 36: (601, 121)}

In [2]:
model = YOLO('yolov8n-pose.pt')

results = 

100%|██████████| 6.51M/6.51M [00:00<00:00, 11.5MB/s]
